## Setup

If you are on Colab run `%tensorflow_version 2.x` to switch to a TensorFlow 2.0 environment. Otherwise, run `pip install tensorflow` or `pip install tensorflow-gpu` (based on your environment). 

## Imports & configuration

In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import class_weight
from wandb.keras import WandbCallback
from ast import literal_eval
from typing import Union
from utils import utils
import tensorflow as tf
import numpy as np
import wandb
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data loading

In [2]:
def load_data(filename:str) -> np.ndarray:
    content = np.load(filename, allow_pickle=True)
    return content

In [3]:
X_train, y_train = load_data('data/X_train.npy'), load_data('data/y_train.npy')
X_test, y_test = load_data('data/X_test.npy'), load_data('data/y_test.npy')

X_train.shape, X_test.shape

((26152,), (6538,))

## Cleaning the paper titles

In [4]:
clean_title = np.vectorize(utils.clean_title)

In [5]:
X_train = clean_title(X_train)
X_test = clean_title(X_test)

In [6]:
# Preview
X_train[:10]

array(['deblurgan blind motion deblurring using conditional adversarial networks',
       'improve satsolving machine learning',
       'training adversarial discriminators crosschannel abnormal event detection crowds',
       'collective stability networks winnertakeall circuits',
       'sample complexity episodic fixedhorizon reinforcement learning',
       'visualizing textual models intext wordaspixel highlighting',
       'prophit causal inverse classification multiple continuously valued treatment policies',
       'sequential dual deep learning shape texture features sketch recognition',
       'notes using determinantal point processes clustering applications text clustering',
       'exactly robust kernel principal component analysis'],
      dtype='<U185')

## Initialize `wandb`

We will be initializing wandb with a different project name this time. We will be setting `sync_tensorboard=True` so that it _automatically_ hosts TensorBoard on my online WandB dashboard. Isn't that cool (TBA)?

In [7]:
def init_wandb(name):
    wandb.init(project='arxiv-project-complex-models', sync_tensorboard=True, name=name)
    config = wandb.config
    return config

Setting the following hyperparameters will allow `wandb` to automatically keep a track of them. 

In [8]:
def init_hyperparams(config):
    config.filter_length = 300
    config.max_words = 3000
    config.maxlen = 300
    config.batch_size = 32
    config.embedding_dims = 30
    config.filters = 10
    config.kernel_size = 3
    config.hidden_dims = 10
    config.epochs = 10
    
    return config

Each time we train a new model we will run the following. 

In [9]:
config = init_wandb("cnn")
config = init_hyperparams(config)

Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


## Tokenization & padding

In [10]:
tokenizer = Tokenizer(num_words=config.max_words, lower=True)
tokenizer.fit_on_texts(X_train)

In [11]:
def get_features(text_sequence: np.ndarray) -> np.ndarray:
    sequences = tokenizer.texts_to_sequences(text_sequence)
    return pad_sequences(sequences, maxlen=config.maxlen)

train_features = get_features(X_train)
test_features = get_features(X_test)

In [12]:
train_features.shape, test_features.shape

((26152, 300), (6538, 300))

In [13]:
y_train[:10]

array(["['cs.CV']", "['cs.AI', 'cs.LO']", "['cs.CV']", "['cs.NE']",
       "['stat.ML', 'cs.AI', 'cs.LG']", "['stat.ML', 'cs.CL', 'cs.LG']",
       "['cs.LG', 'stat.ML']", "['cs.CV']", "['cs.LG']",
       "['cs.LG', 'stat.ML']"], dtype=object)

In [14]:
type(train_features), type(y_train)

(numpy.ndarray, numpy.ndarray)

## Label preprocessing

In [15]:
# Label binarization
list_preprocessed = [literal_eval(i) for i in y_train]
mlb = MultiLabelBinarizer()
y_train_binarized = mlb.fit_transform(list_preprocessed)
mlb.classes_

array(['cs.AI', 'cs.CC', 'cs.CE', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY',
       'cs.DB', 'cs.DS', 'cs.GR', 'cs.GT', 'cs.HC', 'cs.IR', 'cs.IT',
       'cs.LG', 'cs.LO', 'cs.MA', 'cs.MM', 'cs.NE', 'cs.PL', 'cs.RO',
       'cs.SD', 'cs.SE', 'cs.SI', 'math.IT', 'math.OC', 'math.ST',
       'stat.AP', 'stat.CO', 'stat.ME', 'stat.ML', 'stat.TH'],
      dtype=object)

In [16]:
y_train_binarized[:10]

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [17]:
y_train_binarized[0].shape

(32,)

In [18]:
y_test_binarized = mlb.transform([literal_eval(i) for i in y_test])
y_test_binarized[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0,

## Determine the class weights

In [19]:
class_weight = class_weight.compute_sample_weight('balanced', y_train)
class_weight

array([0.03675098, 1.08066116, 0.03675098, ..., 0.03675098, 4.84296296,
       0.08631023])

## More complex models

I will start-off with a CNN-based (Conv1D) architecture and then I will try out LSTM and GRU-based architectures. 

In [20]:
# Helper function to return a compiled CNN-based model
def get_a_cnn_model(config: wandb.wandb_config.Config) -> tf.keras.models.Sequential:
    model = Sequential()
    model.add(Embedding(config.max_words, config.embedding_dims, 
        input_length=config.maxlen))
    model.add(Dropout(0.1))
    model.add(Conv1D(config.filter_length, config.kernel_size, 
        padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    model.add(Dense(32, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
    return model

Pay close attention to the loss function and the metrics. [This blog post](https://www.depends-on-the-definition.com/guide-to-multi-label-classification-with-neural-networks/) helped me a lot to reason why you should use a `binary_crossentropy` loss for a multi-label classification problem. Let's define another function as well for the LSTM and GRU-based models.

In [21]:
# Helper function to return either a LSTM-based or a 
# GRU-based model
def get_a_sequential_model(sequential_layer:Union[tf.keras.layers.LSTM, 
                           tf.keras.layers.GRU], 
                           bidirectional:bool, 
                           config: wandb.wandb_config.Config) -> tf.keras.models.Sequential:
    model = Sequential()
    model.add(Embedding(config.max_words, 20, input_length=config.maxlen))
    if bidirectional:
        model.add(Bidirectional(sequential_layer(10, activation="sigmoid")))
    else:
        model.add(sequential_layer(10, activation="sigmoid"))
    model.add(Dense(32, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    return model

In [22]:
# A helper training script
def train_model(model:tf.keras.models.Sequential,
    config: wandb.wandb_config.Config,
    class_weight=None,
    epochs=config.epochs,
    batch_size=config.batch_size,
    callbacks=None) -> (tf.keras.callbacks.History, str):
    start = time.time()
    history = model.fit(train_features, y_train_binarized,
                        class_weight=class_weight,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=0.1,
                        callbacks=callbacks)
    time_message = 'It took {} seconds'.format(time.time()-start)
    return (history, time_message)

In [23]:
# A helper function to evaluate a model
def get_metrics(model: tf.keras.models.Sequential,
    test_data: tuple) -> (str, str):
    (X_test, y_test) = test_data
    metrics = model.evaluate(X_test, y_test, verbose=None)
    loss = '{}: {}'.format(model.metrics_names[0], metrics[0])
    cat_accuracy = '{}: {}'.format(model.metrics_names[1], metrics[1])
    return(loss, cat_accuracy)

In [24]:
# Define the callbacks
callbacks = [ 
    ReduceLROnPlateau(),
    EarlyStopping(patience=4), 
    ModelCheckpoint(filepath='checkpoints/model-{}.h5'.format(time.time()), save_best_only=True),
    WandbCallback()
]

Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


## CNN-based model

In [27]:
cnn_model = get_a_cnn_model(config)
(history, time_message) = train_model(cnn_model, config, callbacks=callbacks)
print(time_message)

Train on 23536 samples, validate on 2616 samples
Epoch 1/10
23536/23536 [==============================] - 11s 451us/sample - loss: 0.1157 - categorical_accuracy: 0.5338 - val_loss: 0.0831 - val_categorical_accuracy: 0.6892
Epoch 2/10
23536/23536 [==============================] - 4s 168us/sample - loss: 0.0761 - categorical_accuracy: 0.6939 - val_loss: 0.0770 - val_categorical_accuracy: 0.6980
Epoch 3/10
23536/23536 [==============================] - 4s 166us/sample - loss: 0.0701 - categorical_accuracy: 0.7050 - val_loss: 0.0759 - val_categorical_accuracy: 0.7152
Epoch 4/10
23536/23536 [==============================] - 4s 164us/sample - loss: 0.0666 - categorical_accuracy: 0.7223 - val_loss: 0.0765 - val_categorical_accuracy: 0.6793
Epoch 5/10
23536/23536 [==============================] - 4s 165us/sample - loss: 0.0639 - categorical_accuracy: 0.7257 - val_loss: 0.0769 - val_categorical_accuracy: 0.6999
Epoch 6/10
23536/23536 [==============================] - 4s 164us/sample - loss

In [28]:
(loss, categorical_accuracy) = get_metrics(cnn_model, (test_features, y_test_binarized))
print(loss)
print(categorical_accuracy)

loss: 0.07772368787306097
categorical_accuracy: 0.6725298166275024


## Inference

In [25]:
def generate_predictions(model:tf.keras.models.Sequential, article_title: str) -> list:
    labels = []
    
    title = np.array([article_title])
    cleaned_title = clean_title(title)
    tokenized = get_features(cleaned_title)
    
    probabilities = model.predict(tokenized)
    
    probabilities = probabilities.reshape(32,)
    idxs = np.argsort(probabilities)[::-1][:2]
    
    for (i, j) in enumerate(idxs):
        label = "{}: {:.2f}%".format(mlb.classes_[j], probabilities[j] * 100)
        labels.append(label)
        
    return (labels)

In [30]:
print(generate_predictions(cnn_model, 'On the Variance of the Adaptive Learning Rate and Beyond'))

['cs.LG: 89.45%', 'stat.ML: 88.87%']


![](https://i.ibb.co/rZJ0CvN/Screen-Shot-2019-10-31-at-3-01-26-PM.png)

In [31]:
print(generate_predictions(cnn_model, 
        'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'))

['cs.CL: 95.96%', 'cs.LG: 37.99%']


![](https://i.ibb.co/0JJvSBc/Screen-Shot-2019-10-31-at-3-05-05-PM.png)

In [32]:
print(generate_predictions(cnn_model, 
        'MultiFiT: Efficient Multi-lingual Language Model Fine-tuning'))

['cs.CL: 98.14%', 'cs.AI: 8.60%']


![](https://i.ibb.co/1QdcwZj/Screen-Shot-2019-10-31-at-3-09-10-PM.png)

To be able to figure out why the model is giving out additional/erroneous tags in some cases, we can look into the embeddings it is learning and how they correspond to the respective labels. 

Let's now try out other models. 

## GRU

In [34]:
config = init_wandb("gru_non_bi")
config = init_hyperparams(config)

In [35]:
gru_model = get_a_sequential_model(sequential_layer=GRU, bidirectional=False, config=config)
(history, time_message) = train_model(gru_model, config, callbacks=callbacks)
print(time_message)

Train on 23536 samples, validate on 2616 samples
Epoch 1/10
23536/23536 [==============================] - 268s 11ms/sample - loss: 0.2395 - categorical_accuracy: 0.3259 - val_loss: 0.1316 - val_categorical_accuracy: 0.3505
Epoch 2/10
23536/23536 [==============================] - 263s 11ms/sample - loss: 0.1240 - categorical_accuracy: 0.3509 - val_loss: 0.1245 - val_categorical_accuracy: 0.3505
Epoch 3/10
23536/23536 [==============================] - 265s 11ms/sample - loss: 0.1209 - categorical_accuracy: 0.3509 - val_loss: 0.1235 - val_categorical_accuracy: 0.3505
Epoch 4/10
23536/23536 [==============================] - 270s 11ms/sample - loss: 0.1203 - categorical_accuracy: 0.3509 - val_loss: 0.1234 - val_categorical_accuracy: 0.3505
Epoch 5/10
23536/23536 [==============================] - 267s 11ms/sample - loss: 0.1202 - categorical_accuracy: 0.3509 - val_loss: 0.1234 - val_categorical_accuracy: 0.3505
Epoch 6/10
23536/23536 [==============================] - 262s 11ms/sample -

In [36]:
(loss, categorical_accuracy) = get_metrics(gru_model, (test_features, y_test_binarized))
print(loss)
print(categorical_accuracy)

loss: 0.1043574939849144
categorical_accuracy: 0.5711226463317871


In [37]:
print(generate_predictions(gru_model, 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'))

['cs.CL: 34.72%', 'cs.AI: 30.27%']


## LSTM

In [26]:
config = init_wandb("bi_lstm")
config = init_hyperparams(config)

Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


In [27]:
bi_lstm_model = get_a_sequential_model(sequential_layer=LSTM, bidirectional=True, config=config)
(history, time_message) = train_model(bi_lstm_model, config, callbacks=callbacks)
print(time_message)

Train on 23536 samples, validate on 2616 samples
Epoch 1/10


Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


23536/23536 [==============================] - 364s 15ms/sample - loss: 0.2004 - categorical_accuracy: 0.2887 - val_loss: 0.1247 - val_categorical_accuracy: 0.3505
Epoch 2/10
23536/23536 [==============================] - 355s 15ms/sample - loss: 0.1207 - categorical_accuracy: 0.3509 - val_loss: 0.1234 - val_categorical_accuracy: 0.3505
Epoch 3/10
23536/23536 [==============================] - 354s 15ms/sample - loss: 0.1194 - categorical_accuracy: 0.3552 - val_loss: 0.1207 - val_categorical_accuracy: 0.3949
Epoch 4/10
23536/23536 [==============================] - 354s 15ms/sample - loss: 0.1134 - categorical_accuracy: 0.4533 - val_loss: 0.1121 - val_categorical_accuracy: 0.5023
Epoch 5/10
23536/23536 [==============================] - 353s 15ms/sample - loss: 0.1024 - categorical_accuracy: 0.5502 - val_loss: 0.1011 - val_categorical_accuracy: 0.5837
Epoch 6/10
23536/23536 [==============================] - 367s 16ms/sample - loss: 0.0914 - categorical_accuracy: 0.6374 - val_loss: 0.0

In [28]:
(loss, categorical_accuracy) = get_metrics(bi_lstm_model, (test_features, y_test_binarized))
print(loss)
print(categorical_accuracy)

loss: 0.08260532424797447
categorical_accuracy: 0.6708473563194275


In [29]:
print(generate_predictions(bi_lstm_model, 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'))

['cs.CL: 84.36%', 'cs.AI: 10.34%']


## Visualization of the embeddings (learned by the CNN-based model)

In [ ]:
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
def serialize_embeddings(model:tf.keras.models.Sequential) -> str:
    # Get the weights of the first layer (Embedding layer)
    e = model.layers[0]
    weights = e.get_weights()[0]
    # Serialize the weights in .tsv format
    out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
    out_m = io.open('meta.tsv', 'w', encoding='utf-8')
    for word_num in range(1, config.max_words):
        word = reverse_word_index[word_num]
        embeddings = weights[word_num]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    out_v.close()
    out_m.close()
    return 'Embeddings have been serialized'

In [ ]:
print(serialize_embeddings(cnn_model))

Here's a screencast of the projection of the Embedding matrix: https://www.loom.com/share/88eaf892c02a4ba392b00ff376686e0d